## YOLO: You Only Look Once
- yolov3.cfg 
- yolov3.weights
- classes.txt

## YOLO Theory
- https://stackoverflow.com/questions/50575301/yolo-object-detection-how-does-the-algorithm-predict-bounding-boxes-larger-than
- https://www.kaggle.com/utkarshxy/object-detection-with-yolo-complete-theory-5mins
- https://www.youtube.com/watch?v=1LCb1PVqzeY&t=0s



## Import Library

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Configuration YOLO 
readNet(model[, config[, framework]]) -> retval
   * @brief Read deep learning network represented in one of the supported formats.
   * @param[in] model Binary file contains trained weights. The following file
   *                  extensions are expected for models from different frameworks:
   *                  * `*.caffemodel` (Caffe, http://caffe.berkeleyvision.org/)
   *                  * `*.pb` (TensorFlow, https://www.tensorflow.org/)
   *                  * `*.t7` | `*.net` (Torch, http://torch.ch/)
   *                  * `*.weights` (Darknet, https://pjreddie.com/darknet/)
   *                  * `*.bin` (DLDT, https://software.intel.com/openvino-toolkit)
   * @param[in] config Text file contains network configuration. It could be a
   *                   file with the following extensions:
   *                  * `*.prototxt` (Caffe, http://caffe.berkeleyvision.org/)
   *                  * `*.pbtxt` (TensorFlow, https://www.tensorflow.org/)
   *                  * `*.cfg` (Darknet, https://pjreddie.com/darknet/)
   *                  * `*.xml` (DLDT, https://software.intel.com/openvino-

In [ ]:
net=cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
classes=[]
with open('classes.txt','r') as f:
    classes=f.read().splitlines()

In [3]:
print('Total classes: ', len(classes))
print(classes)

Total classes:  80
['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


## Import Image for object Detection

In [ ]:
image=cv2.imread('people.jpg')
cv2.imshow('Input Image: ',image)
cv2.waitKey()
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to target thread (0x555e15df0b20)

QObject::moveToThread: Current thread (0x555e15df0b20) is not the object's thread (0x555e164f2450).
Cannot move to tar

In [ ]:
print('Image shape: ',image.shape)
height,width,channel=image.shape
print(height)
print(width)

In [ ]:
yolo_input_image=np.copy(image)
yolo_input_image.shape

## Blob from Image
blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval

In [ ]:
blob=cv2.dnn.blobFromImage(yolo_input_image,1/255,(416,416),(0,0,0),swapRB=True,crop=False)

In [ ]:
blob.shape

In [ ]:
for b in blob:
    print(b.shape)
    # indicate that there are 3 channel and each channel image size is: (image.height x image.width)

In [ ]:
for b in blob:
    for channel,img_blob in enumerate(b):
        cv2.imshow(str(channel),img_blob)
cv2.waitKey()
cv2.destroyAllWindows()

## Initialize YOLO with Blobs

In [ ]:
net.setInput(blob)

In [ ]:
output_layers_name=net.getUnconnectedOutLayersNames()
output_layers_name

In [ ]:
# Runs forward pass to compute output of layer with name @p outputName.
layerOutputs=net.forward(output_layers_name)

In [ ]:
len(layerOutputs)

In [ ]:
layerOutputs[0][0][0]

## Define Box Parameter, Confidence and Class ids

In [ ]:
boxes=[]
confidences=[]
class_ids=[]

In [ ]:
# First four parameters are: center x, center y, box width , box height
# Others parametre indicate the predicted class id
layerOutputs[0][0]

In [ ]:
for output in layerOutputs:
    for detection in output:
        scores=detection[5:]
        class_id=np.argmax(scores)
        confidence=scores[class_id]
        if confidence>0.5:
            center_x=int(detection[0]*width)
            center_y=int(detection[1]*height)
            w=int(detection[2]*width)
            h=int(detection[3]*height)
            
            x=int(center_x -(w/2))
            y=int(center_y -(h/2))
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            

In [ ]:
class_ids

In [ ]:
print('Box found: ',len(boxes))
print(boxes)

## Non Maximum Supression 
NMSBoxes(bboxes, scores, score_threshold, nms_threshold[, eta[, top_k]]) -> indices

If the two boxes are correlated with each other. we have to choose one box rather than two.In this scenario, <b>NMSBoxes</b> removes the lower scores bounding box if two boxes are mixed with each others

In [ ]:
type(confidences[0])

In [ ]:
type(boxes[0][0])

In [ ]:
indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.3,0.4)
indexes

In [ ]:
print('After Non Maximum Suppression (NMS): ',len(indexes))
indexes.flatten()

## Draw the Bounding Box on Image 

In [ ]:
font=cv2.FONT_HERSHEY_PLAIN
colors=np.random.uniform(0,255,size=(len(boxes),3))

In [ ]:
print(colors.shape)
colors

In [ ]:
if len(indexes)>0:
    for i in indexes.flatten():
        x,y,w,h=boxes[i]
        label=str(classes[class_ids[i]])
        color=colors[i]
        confidence=str(round(confidences[i],2))
        
        cv2.rectangle(yolo_input_image,(x,y),(x+w,y+h),color,2)
        cv2.putText(yolo_input_image,label+' '+confidence,(x,y+20),font,2,(0,0,255),2 )
cv2.imshow('YOLO Input',image)
cv2.imshow('YOLO Output',yolo_input_image)
cv2.waitKey()
cv2.destroyAllWindows()